In [ ]:
#Imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
def print_mono_image(image, title = None):
    plt.imshow(image, cmap='gray')
    if (title is not None):
        plt.title(title)
    plt.axis('off')
    plt.show()



In [ ]:
# Leitura e conversão da imagem para cinza 
image_path = '../imagens/baboon_monocromatica.png'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

In [ ]:
# Calcula a Transformada Discreta de Fourier bidimensional
fft_image = np.fft.fft2(image)

# Deslocar o centro de frequência
fft_centered = np.fft.fftshift(fft_image)

# Extrair a magnitude de cada coeficiente
abs_image = np.abs(fft_centered)
magnitude = np.log1p(abs_image)
print_mono_image(magnitude)

In [ ]:
# Criação de máscaras:
h, w = image.shape
center_row, center_column = h // 2, w // 2
r, R = 30, 80
# Passa baixa
mask_pb = np.zeros_like(image, dtype=float)
# Passa alta
mask_pa = np.zeros_like(image, dtype=float)
# Passa faixa
mask_pf = np.zeros_like(image, dtype=float)
# Rejeita faixa
mask_rf = np.zeros_like(image, dtype=float)

y, x = np.ogrid[:h, :w]

d = np.sqrt(np.power(y - center_row, 2) + np.power(x - center_column, 2))

# Passa alta
mask_pb[d <= r] = 1
mask_pa[d >= R] = 1
mask_pf[(d >= r) & (d <= R)] = 1
mask_rf[(d <= r) | (d >= R)] = 1

In [ ]:
## Resultado de como são as mascaras
dict_filtros = {
    'Espectro Filtrado (Passa-baixa)': mask_pb,
    'Espectro Filtrado (Passa-alta)': mask_pa,
    'Espectro Filtrado (Passa-faixa)': mask_pf,
    'Espectro Filtrado (Rejeita-faixa)': mask_rf,
}
for title, mask in dict_filtros.items():
    plt.figure(figsize=(4,4))
    plt.imshow(magnitude * mask, cmap='gray', vmin=magnitude.min(), vmax=magnitude.max(),
            interpolation='nearest')
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
fft_pb = fft_centered * mask_pb
fft_pa = fft_centered * mask_pa
fft_pf = fft_centered * mask_pf
fft_rf = fft_centered * mask_rf

In [ ]:
def reconstruct_image(fft_spectrum):
    # reposiciona o componente DC para onde a IFFT espera
    spectrum_shifted_pb = np.fft.ifftshift(fft_spectrum)
    # converte de volta para o domínio espacial
    img_back = np.fft.ifft2(spectrum_shifted_pb)
    # extrai a parte real da matriz complexa resultante
    img_real = np.real(img_back)
    # garante valores não-negativos antes da normalização
    img_abs = np.abs(img_real)
    img_norm = 255 * (img_abs - img_abs.min()) / (img_abs.max() - img_abs.min())
    img_norm_uint8 = img_norm.astype(np.uint8)
    return img_norm_uint8

reconstructed_images = {
    'Reconstrução (IFFT) — Filtro Passa–Baixa': fft_pb,
    'Reconstrução (IFFT) — Filtro Alta': fft_pa,
    'Reconstrução (IFFT) — Filtro Passa–Faixa': fft_pf,
    'Reconstrução (IFFT) — Filtro Rejeita–Faixa': fft_rf,
}
for title, fft_spectrum in reconstructed_images.items():
    img_norm_uint8 = reconstruct_image(fft_spectrum)
    print_mono_image(img_norm_uint8, title)

In [ ]:
fft_comp = fft_centered.copy()
Threshold = 200000
mask_threshold = np.abs(fft_comp) < Threshold
fft_comp[mask_threshold] = 0
img_compressed = reconstruct_image(fft_comp)
print_mono_image(img_compressed)
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.hist(image.ravel(), bins=256, range=(0,255))
plt.title('Histograma – Original')
plt.subplot(1,2,2)
plt.hist(img_compressed.ravel(), bins=256, range=(0,255))
plt.title('Histograma – Comprimida')
plt.show()

: 